In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizerFast
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

def train_model(df, intents, model, callbacks, batch_size, epochs):
    # Load the BERT tokenizer
    tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

    # Encode the text data
    input_encodings = tokenizer(df['text'].tolist(), truncation=True, padding=True, max_length=512)

    # Create binary labels for each intent
    labels = df[intents].values

    # Split the data into training and validation sets
    train_inputs, val_inputs, train_labels, val_labels = train_test_split(input_encodings['input_ids'], labels, test_size=0.2)

    # Convert the data to TensorFlow Datasets
    train_dataset = tf.data.Dataset.from_tensor_slices((train_inputs, train_labels)).shuffle(10000).batch(batch_size)
    val_dataset = tf.data.Dataset.from_tensor_slices((val_inputs, val_labels)).batch(batch_size)

    # Compile the model with a optimizer, loss, and metrics
    model.compile(optimizer=Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
                  loss='binary_crossentropy',  # Using binary_crossentropy for multi-label classification
                  metrics=['accuracy'])

    # Train the model
    model.fit(train_dataset, epochs=epochs, validation_data=val_dataset, callbacks=callbacks)
    
    return model
